In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
! pip install transformers
! pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import keras
import pandas as pd
import tensorflow as tf
import numpy as np
from datasets import Dataset,load_dataset
from transformers import AutoTokenizer,TFAutoModelForSequenceClassification

In [4]:
df = pd.read_csv("/content/drive/MyDrive/ara_sent/data.csv",encoding="utf-8")
df.sample(5)

,text,polarity
3372,افضل المطاعم البنانيه مطعم لبناني غني التعريف ...,1
7913,خدمه التوصيل ميزه جدا وليس لسرعتها لبطءها الشد...,0
3494,مطعم وكافيه السلطان احمد مطعم متاز جدا ومكان ه...,1
4109,الافضل افضل مطعم سيوه الطعام راءع الخدمه جيده ...,1
5345,ازعاج خدمه سيءه لايوجد مكان لصلاه اكل متغير دس...,0


In [5]:
X=df['text'].values
y=df['polarity'].values
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=60)

In [6]:
train_df = Dataset.from_dict({'text': X_train, 'label': y_train})
test_df = Dataset.from_dict({'text': X_test, 'label': y_test})

In [7]:
train_df

Dataset({
    features: ['text', 'label'],
    num_rows: 7320
})

In [25]:
max_len = 200 # Maximum length of each sequence
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-multilingual-cased")
# Define the preprocessing function
def preprocess(examples):
    # Tokenize the text using the tokenizer
    encodings = tokenizer.encode_plus(examples["text"], add_special_tokens=True, max_length=max_len, truncation=True, padding='max_length')
    return encodings

In [9]:
train_dataset = train_df.map(preprocess)
test_dataset = test_df.map(preprocess)

Map:   0%|          | 0/7320 [00:00<?, ? examples/s]

Map:   0%|          | 0/2440 [00:00<?, ? examples/s]

In [10]:
id2label = {0: "NEGATIVE", 1: "POSITIVE"}

In [27]:
model = TFAutoModelForSequenceClassification.from_pretrained("distilbert-base-multilingual-cased",num_labels=2,id2label=id2label)

Some layers from the model checkpoint at distilbert-base-multilingual-cased were not used when initializing TFDistilBertForSequenceClassification: ['activation_13', 'vocab_transform', 'vocab_layer_norm', 'vocab_projector']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['classifier', 'dropout_77', 'pre_classifier']
You should probably TRAIN this model on a down-stream ta

In [12]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

tf_train_set = model.prepare_tf_dataset(
    train_dataset,
    shuffle=True,
    batch_size=8,
    collate_fn=data_collator,
)

tf_test_set = model.prepare_tf_dataset(
    test_dataset,
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/datasets/arrow_dataset.py:388: FutureWarning: The output of `to_tf_dataset` will change when a passing single element list for `labels` or `columns` in the next datasets version. To return a tuple structure rather than dict, pass a single string.
Old behaviour: columns=['a'], labels=['labels'] -> (tf.Tensor, tf.Tensor)  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor)  
New behaviour: columns=['a'],labels=['labels'] -> ({'a': tf.Tensor}, {'labels': tf.Tensor})  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor) 
  warnings.warn(


In [19]:
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['BinaryAccuracy'])

In [20]:
model.evaluate(tf_test_set)

152/152 [==============================] - 45s 246ms/step - loss: 0.4567 - binary_accuracy: 0.7880


[0.4566507339477539, 0.7880345582962036]

In [15]:
history = model.fit(tf_train_set, validation_data=tf_test_set, epochs=5)

Epoch 1/6
915/915 [==============================] - 444s 432ms/step - loss: 0.7176 - accuracy: 0.4926 - val_loss: 0.6660 - val_accuracy: 0.6077
Epoch 2/6
915/915 [==============================] - 373s 408ms/step - loss: 0.6529 - accuracy: 0.4676 - val_loss: 0.6622 - val_accuracy: 0.6151
Epoch 3/6
915/915 [==============================] - 365s 399ms/step - loss: 0.5962 - accuracy: 0.4432 - val_loss: 0.5033 - val_accuracy: 0.7126
Epoch 4/6
915/915 [==============================] - 364s 398ms/step - loss: 0.6440 - accuracy: 0.5150 - val_loss: 0.7397 - val_accuracy: 0.6357
Epoch 5/6
915/915 [==============================] - 364s 398ms/step - loss: 0.6273 - accuracy: 0.4396 - val_loss: 0.5529 - val_accuracy: 0.2381
Epoch 6/6
915/915 [==============================] - 370s 404ms/step - loss: 0.5963 - accuracy: 0.5295 - val_loss: 0.4572 - val_accuracy: 0.3890


In [16]:
model.evaluate(tf_test_set)

152/152 [==============================] - 36s 235ms/step - loss: 0.4575 - accuracy: 0.3894


[0.4575406610965729, 0.3893914520740509]

In [24]:
#txt = ["خدمة زباله","وحش جدا المكان محدش يروحه","انصح بالفيلم ده كويس ","انا حزين جدا","سعيد جدا الان "]
txt=" انصح بالفيلم ده كويس"
inputs = tokenizer(txt, add_special_tokens=True, max_length=max_len, truncation=True, padding='max_length',return_tensors="tf")
logits = model(**inputs).logits
predicted_class_id = int(tf.math.argmax(logits, axis=-1)[0])
id2label[predicted_class_id]

'NEGATIVE'

In [ ]:
for i in range(len(txt)):
  print(f'{txt[i]} : { "ايجابي"if predictions[i][0]>0.5 else "سلبي" }')


In [ ]:
#from huggingface_hub import notebook_login
#notebook_login()
#from transformers.keras_callbacks import PushToHubCallback
#push_to_hub_callback = PushToHubCallback( output_dir="model", tokenizer=tokenizer)
#,callbacks=push_to_hub_callback

In [ ]:
#predicted_labels = tf.argmax(predictions, axis=1)